In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import nltk
import requests
import random
import math
import pathlib 
import modules.hein_scraping_functions

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait

In [2]:
# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("https://scholar.google.com/scholar?hl=en&as_sdt=5%2C36&sciodt=0%2C36&cites=17432944610365151854&scipsc=&q=%22Australian+Coastal+and+Marine+Law%22&oq=")


In [11]:
# Load the dataset from the input directory
paper_df = pd.read_excel(input_path / "google_scholar_paper_list.xlsx", sheet_name='Sheet1')
data = paper_df

In [22]:
def search_google_scholar(search_text, driver):

    driver.get("https://scholar.google.com/scholar?hl=en&as_sdt=5%2C36&sciodt=0%2C36&cites=17432944610365151854&scipsc=&q=%22Australian+Coastal+and+Marine+Law%22&oq=")
    
    # Make sure the webpage has loaded
    webpage_wait('//*[@id="gs_bdy_ccl"]', driver)

    # Enter the search text
    full_text = driver.find_element_by_xpath('//*[@id="gs_hdr_tsi"]')
    full_text.clear()
    full_text.send_keys(search_text)

    # Click the search button
    search = driver.find_element_by_xpath('//*[@id="gs_hdr_tsb"]/span/span[1]').click()
    # Look for the number of results pannel
    results_count_elm = driver.find_element_by_xpath('//*[@id="gs_res_ccl_mid"]/div/div/div[3]/a[3]')
    results_count_text = results_count_elm.text
    # This only matches integers up to 999,999, but I doubt that will be a problem.
    match = re.search(r"^Cited by (0|[1-9]\d{0,2},?\d*)", results_count_text)
    if match == None:
        result_count = 0
    else:
        result_count = match.group(1)
    
    driver.get("http://amazon.com")
    time.sleep(3)
    driver.get("http://facebook.com")
    return result_count


In [23]:
# Search for each string and save the results to the dataset
data_subset = data.iloc[0:100, :]
data_subset["google scholar cite count"] = data_subset.apply(lambda x: search_google_scholar(x["google scholar search"], driver), axis = 1)

<ipython-input-23-ad3400c9dbaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset["google scholar cite count"] = data_subset.apply(lambda x: search_google_scholar(x["google scholar search"], driver), axis = 1)


In [24]:
# Export to Excel
data_subset.to_excel(intr_path / "_google_scholar_cites_df.xlsx", index = False)